In [1]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect("goLance_data.db")
%sql conn --alias duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

jobs table

In [2]:
%%sql
select * from jobs limit 5

Running query in 'duckdb'

job_id,category,posted_at,filled_at,client_id,status
1,Mobile Development,2024-10-02 16:24:04.670261,2024-10-16 01:50:00.670261,29,filled
2,Marketing,2024-02-21 06:34:39.799722,2024-02-26 06:59:34.799722,58,filled
3,Mobile Development,2024-05-22 14:10:49.592034,None,190,expired
4,Web Development,2024-05-03 15:57:04.336128,2024-05-05 03:40:02.336128,152,filled
5,Web Development,2024-11-07 03:04:09.676973,2024-12-06 09:28:18.676973,56,filled


In [3]:
%%sql
select * from jobs limit 5

Running query in 'duckdb'

job_id,category,posted_at,filled_at,client_id,status
1,Mobile Development,2024-10-02 16:24:04.670261,2024-10-16 01:50:00.670261,29,filled
2,Marketing,2024-02-21 06:34:39.799722,2024-02-26 06:59:34.799722,58,filled
3,Mobile Development,2024-05-22 14:10:49.592034,None,190,expired
4,Web Development,2024-05-03 15:57:04.336128,2024-05-05 03:40:02.336128,152,filled
5,Web Development,2024-11-07 03:04:09.676973,2024-12-06 09:28:18.676973,56,filled


1. Median Time to Fill a Job for Each Job Category

In [5]:
%%sql
select 
    
    category,
    median(filled_at - posted_at) as median_fill_time

from jobs

where status = 'filled'

group by category
order by category

Running query in 'duckdb'

category,median_fill_time
Data Science,"18 days, 4:38:28"
Graphic Design,"15 days, 4:27:23"
Marketing,"16 days, 4:01:26"
Mobile Development,"16 days, 19:57:19"
Web Development,"17 days, 11:13:03"
Writing,"13 days, 13:31:43"


2. Job Application Conversion Rate

In [17]:
%%sql
select
    
    job_id,
    round(
        100 * sum(case when status = 'accepted' then 1 else 0 end) / count(*), 2
    ) as conversion_rate

from freelancer_applications
group by job_id

Running query in 'duckdb'

job_id,conversion_rate
1,7.14
2,4.35
3,0.0
4,25.0
5,50.0
6,20.0
7,5.26
8,0.0
9,3.33
10,6.25


3. Top 3 Job Categories with the Lowest Fulfillment Rate

In [24]:
%%sql
select

  category,
  round(100.0 * sum(case when status = 'filled' then 1 else 0 end) / count(*)) as fulfillment_rate

from jobs
group by category
order by fulfillment_rate asc
limit 3

Running query in 'duckdb'

category,fulfillment_rate
Writing,53.0
Data Science,55.0
Web Development,59.0
Graphic Design,61.0
Mobile Development,64.0
Marketing,64.0
